<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/Project/No%20Name/Project_BigData_NoName.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJECT: Amazon Book Reviews**

**Group 7**: No Name  


| Name | Matric No. |
| --- | ----------- |
| Madina Suraya Binti Zharin | A20EC0203 |
| Nur Izzah Mardhiah binti Rashidi | A20EC0116 |
| Tan Yong Sheng | 	A20EC0157 |
| Chloe Racquelmae Kennedy | A20EC0026 |  

**Amazon Book Reviews** dataset contain feedback about 3M user on 212404 unique books. The dataset contains product reviews and metadata from Amazon, including 142.8 million reviews spanning May 1996 - July 2014.This dataset was found from Kaggle and its size is 2.86 GB.

src: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=Books_rating.csv

The dataset contains 3000000 records and 10 columns which are:

| Features | Description |
| --- | ----------- |
| id | The Id of Book |
| Title | 	Book Title |
| Price | The price of Book |
| User_id | 	Id of the user who rates the book |
| profileName | 	Name of the user who rates the book |
| review/helpfulness | helpfulness rating of the review, e.g. 2/3 |  
| review/score | rating from 0 to 5 for the book |  
| review/time | time of given the review |  
| review/summary | the summary of a text review |  
| review/text | the full text of a review |  

## Load Dataset

First, mount drive into google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
df =pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/Books_rating.csv")
pd.set_option('display.max_columns', None)
df.head(5)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


## Data Preparation and Cleaning

**Sampling** - Reduce memory usage

In [ ]:
n = sum(1 for line in open("/content/drive/MyDrive/Colab Notebooks/Project/Books_rating.csv"))-1  # Calculate number of rows in file
s = n//10  # sample size of 10%
skip = sorted(random.sample(range(1, n+1), n-s))  # n+1 to compensate for header 
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/Books_rating.csv", skiprows=skip)
pd.set_option('display.max_columns', None)
df.head(5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Id                  300000 non-null  object 
 1   Title               299981 non-null  object 
 2   Price               48318 non-null   float64
 3   User_id             243913 non-null  object 
 4   profileName         243905 non-null  object 
 5   review/helpfulness  300000 non-null  object 
 6   review/score        300000 non-null  float64
 7   review/time         300000 non-null  int64  
 8   review/summary      299997 non-null  object 
 9   review/text         299999 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 22.9+ MB


After sampling, the memory usage decrease 90% from 228.9+ MB to 22.9+ MB.

**Check duplicated rows**

In [ ]:
df.shape

(300000, 10)

In [ ]:
duplicate_rows_data = df[df.duplicated()]
print("Number of duplicate rows: ", duplicate_rows_data.shape)

Number of duplicate rows:  (97, 10)


There are 97 duplicated rows.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(299903, 10)

**Check null columns and rows**

In [ ]:
df.isna().sum()

Id                         0
Title                     19
Price                 251591
User_id                56062
profileName            56070
review/helpfulness         0
review/score               0
review/time                0
review/summary             3
review/text                1
dtype: int64

- Since there are 90% records in **Price** column are null, we drop the column.
- Drop rows with null in **Title**, **review/summary** and **review/text**.
- Since only 20% records of **User_id** and **ProfileName** are null, we keep the columns and null value are acceptable.
- Make sure that there are only a maximum of 2 null value are allowed in each record.

In [ ]:
df = df.drop(['Price'], axis=1)

In [ ]:
df = df.dropna(axis=0, subset=['Title', 'review/summary', 'review/text'])

In [ ]:
df = df.dropna(thresh=2,axis=0)

In [ ]:
df.isna().sum()

Id                        0
Title                     0
User_id               56057
profileName           56065
review/helpfulness        0
review/score              0
review/time               0
review/summary            0
review/text               0
dtype: int64

**Remove not applicable column**

In [ ]:
df = df.drop(['review/time'], axis=1)